In [ ]:
#predicting the price of the GE stock, using open, high,low close and adj close

import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [ ]:
df=pd.read_csv('/content/GE (1).csv')


In [ ]:
# convert the date in the data to datetime
train_dates=pd.to_datetime(df['Date'])

#variables for training
cols=list(df)[1:7]
print(df.head())
print(df.dtypes)
df.isnull().values.any()
df_fortraining=df[cols].astype(float)
print(df)

         Date       Open       High        Low      Close  Adj Close   Volume
0  2022-11-29  66.939888  67.072601  66.323189  66.869637  66.645943  5750409
1  2022-11-30  66.854019  67.408272  64.941452  67.111633  66.887131  9301085
2  2022-12-01  67.298988  67.611244  66.252930  66.557381  66.334732  5432337
3  2022-12-02  66.018738  67.876656  65.893829  67.822014  67.595131  5739392
4  2022-12-05  67.010147  67.174080  66.057770  66.088989  65.867905  6040427
Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object
           Date        Open        High         Low       Close   Adj Close  \
0    2022-11-29   66.939888   67.072601   66.323189   66.869637   66.645943   
1    2022-11-30   66.854019   67.408272   64.941452   67.111633   66.887131   
2    2022-12-01   67.298988   67.611244   66.252930   66.557381   66.334732   
3    2022-12-02   66.018738   67.876656   65.893829   6

In [ ]:
print(train_dates)

0     2022-11-29
1     2022-11-30
2     2022-12-01
3     2022-12-02
4     2022-12-05
         ...    
246   2023-11-21
247   2023-11-22
248   2023-11-24
249   2023-11-27
250   2023-11-28
Name: Date, Length: 251, dtype: datetime64[ns]


In [ ]:
#the above converts all the collumns to float, removes volume but lets include it so no need to do this
#df_for_training=df[cols].astype(float)
#print(df_for_training.head())
#look at last 5000 data points

#df_for_plot=df_for_training.tail(5000)
#df_for_plot.plot.line()

In [ ]:
#STANDARDIZE DATA
# later on to get original data, you have to do inverse trasform which will accept 5 columns, where you have to do columns by columns so you can apply it to one collumns and transform to all becasue the data is similar
scaler=StandardScaler()
scaler=scaler.fit(df_fortraining)
df_scaled=scaler.transform(df_fortraining)

In [ ]:
print(df_scaled)
print(df.shape[1])

[[-1.92106997 -1.98042884 -1.91256061 -1.95046977 -1.95112655 -0.07565192]
 [-1.92635711 -1.95977077 -1.99807454 -1.93545012 -1.93620312  1.11478691]
 [-1.89895942 -1.94727935 -1.91690885 -1.96985015 -1.97038263 -0.18229225]
 ...
 [ 1.34472352  1.29839982  1.38962405  1.34524084  1.34828179 -1.4732658 ]
 [ 1.32748295  1.29655367  1.36734411  1.34524084  1.34828179 -0.93746915]
 [ 1.34410768  1.28362979  1.32030856  1.27572708  1.27898198 -0.8322611 ]]
7


In [ ]:
trainX=[]
trainY=[]

n_future=1 # number of days we want to predict
n_past=14 # number of days in past we want to use to make prediction
#apply - if you take the first value and want to take 10 samples, so first to 10th is inputs and 11th is the output

#iteration 1:
# for loop goes 14 to end of time series: each iteration, you go i-past so for iteation 1 its 14-14=0, to 14, which means 0 to 13 is 14 days then 14 is the 15th day
for i in range(n_past, len(df_scaled) - n_future +1):
    trainX.append(df_scaled[i - n_past:i, 0:df_scaled.shape[1]])
    trainY.append(df_scaled[i + n_future - 1:i + n_future, 0])


In [ ]:
#convert to numpy array - which will change dimensions cause trainX is a list of lists rn
print(len(trainX))
print(len(trainY))
trainX,trainY=np.array(trainX),np.array(trainY)
print(trainX.shape)
print(trainY.shape)
#will be 14 less becasue of the 14 window, the 6 is the number of inputs features, 237 becasue you have 237 windows of size 14 in the entire dataset

237
237
(237, 14, 6)
(237, 1)


In [ ]:
#model
model = Sequential()
#you have return sequences=true becasue you want this lstm to return a sequence, so it can go to the next LSTM becasue this is a stacked model
#trainX.shape[1]=14 becasue this gives you the 14 windows and then trainX.shape[2]=6 becasue of the inputs
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
#return sequence = false becasue you are going to dropout so dont need the sequence
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

#using mse becasue these are numbers so dont need to do entropy
model.compile(optimizer='adam',loss='mse')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 14, 64)            18176     
                                                                 
 lstm_8 (LSTM)               (None, 14, 32)            12416     
                                                                 
 lstm_9 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 38945 (152.13 KB)
Trainable params: 38945 (152.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(trainX, trainY, epochs=5, batch_size=16, validation_split=0.1, verbose=1)


Epoch 1/5
14/14 [==============================] - 4s 111ms/step - loss: 0.4791 - val_loss: 0.3100
Epoch 2/5
14/14 [==============================] - 1s 99ms/step - loss: 0.1025 - val_loss: 0.1385
Epoch 3/5
14/14 [==============================] - 1s 94ms/step - loss: 0.0729 - val_loss: 0.0862
Epoch 4/5
14/14 [==============================] - 1s 52ms/step - loss: 0.0575 - val_loss: 0.0837
Epoch 5/5
14/14 [==============================] - 1s 51ms/step - loss: 0.0470 - val_loss: 0.0541


In [ ]:
#forecast

#Libraries that will help us extract only business days in the US.
#Otherwise our dates would be wrong when we look back (or forward).
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

#forecasting 90 days now
n_futures=90

#have to extract the correct dates
#train_dates[-1] is the current date or last date in the data, going till 90 days past
forecast_period_dates = pd.date_range(list(train_dates)[-1], periods=n_futures, freq='1d').tolist()
print(forecast_period_dates)

[Timestamp('2023-11-28 00:00:00', freq='D'), Timestamp('2023-11-29 00:00:00', freq='D'), Timestamp('2023-11-30 00:00:00', freq='D'), Timestamp('2023-12-01 00:00:00', freq='D'), Timestamp('2023-12-02 00:00:00', freq='D'), Timestamp('2023-12-03 00:00:00', freq='D'), Timestamp('2023-12-04 00:00:00', freq='D'), Timestamp('2023-12-05 00:00:00', freq='D'), Timestamp('2023-12-06 00:00:00', freq='D'), Timestamp('2023-12-07 00:00:00', freq='D'), Timestamp('2023-12-08 00:00:00', freq='D'), Timestamp('2023-12-09 00:00:00', freq='D'), Timestamp('2023-12-10 00:00:00', freq='D'), Timestamp('2023-12-11 00:00:00', freq='D'), Timestamp('2023-12-12 00:00:00', freq='D'), Timestamp('2023-12-13 00:00:00', freq='D'), Timestamp('2023-12-14 00:00:00', freq='D'), Timestamp('2023-12-15 00:00:00', freq='D'), Timestamp('2023-12-16 00:00:00', freq='D'), Timestamp('2023-12-17 00:00:00', freq='D'), Timestamp('2023-12-18 00:00:00', freq='D'), Timestamp('2023-12-19 00:00:00', freq='D'), Timestamp('2023-12-20 00:00:00'

In [ ]:
#predict
# use [-n_futures] becasue this is -90 which means take 90th last day as the starting date for the sequence then that day till the end of dataset(current date) to get 91st which is the output 1(1st day of 90 days or current date +1) then take the last 90th plus one so the 89th last day till the current date +1, which will predict the current day +2, do this until you are using last part of dataset(ie. current date) to current date+89 to get the current+90th day
forecast=model.predict(trainX[-n_futures:])

3/3 [==============================] - 0s 10ms/step


In [ ]:
#but the above gives you the output in a scaled form so you have to convert this to an unscaled form

#Perform inverse transformation to rescale back to original range
#Since we used 5 variables for transform, the inverse expects same dimensions
#Therefore, let us copy our values 5 times and discard them after inverse transform
prediction_copies = np.repeat(forecast, df_fortraining.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(prediction_copies)[:,0]
print(y_pred_future)

[110.44469  110.425476 110.6982   111.22177  111.87871  112.53211
 113.0619   113.35655  113.50618  113.61176  113.63371  113.80407
 113.97671  114.03882  114.00722  114.107155 113.71554  113.77272
 113.80174  113.659164 113.40517  113.310974 113.25954  113.24437
 113.18021  113.02209  113.03307  113.059    113.09857  113.16015
 113.260124 113.32395  113.323685 113.35933  113.39972  113.37979
 113.45912  113.58153  113.69347  113.76426  113.98707  114.27776
 114.496925 114.48047  114.271545 114.06359  113.730286 113.38358
 113.11277  112.83569  112.38741  111.768524 111.24825  110.77324
 110.40892  110.141075 110.02681  110.21551  110.45137  110.516754
 110.48695  110.43175  110.21101  109.971115 109.5599   109.23253
 109.10917  109.24067  109.21492  109.12884  109.09396  108.979485
 108.66121  108.48202  108.46872  108.59588  108.91662  109.30399
 109.79604  110.41389  110.74488  111.3459   112.071434 112.90739
 113.80511  114.761246 115.73749  116.72019  117.73859  118.506294]


In [ ]:
print(forecast_period_dates)

[Timestamp('2023-11-28 00:00:00', freq='D'), Timestamp('2023-11-29 00:00:00', freq='D'), Timestamp('2023-11-30 00:00:00', freq='D'), Timestamp('2023-12-01 00:00:00', freq='D'), Timestamp('2023-12-02 00:00:00', freq='D'), Timestamp('2023-12-03 00:00:00', freq='D'), Timestamp('2023-12-04 00:00:00', freq='D'), Timestamp('2023-12-05 00:00:00', freq='D'), Timestamp('2023-12-06 00:00:00', freq='D'), Timestamp('2023-12-07 00:00:00', freq='D'), Timestamp('2023-12-08 00:00:00', freq='D'), Timestamp('2023-12-09 00:00:00', freq='D'), Timestamp('2023-12-10 00:00:00', freq='D'), Timestamp('2023-12-11 00:00:00', freq='D'), Timestamp('2023-12-12 00:00:00', freq='D'), Timestamp('2023-12-13 00:00:00', freq='D'), Timestamp('2023-12-14 00:00:00', freq='D'), Timestamp('2023-12-15 00:00:00', freq='D'), Timestamp('2023-12-16 00:00:00', freq='D'), Timestamp('2023-12-17 00:00:00', freq='D'), Timestamp('2023-12-18 00:00:00', freq='D'), Timestamp('2023-12-19 00:00:00', freq='D'), Timestamp('2023-12-20 00:00:00'

In [ ]:
#attach above to date and plot
# Convert timestamp to date
forecast_dates = []
for time_i in forecast_period_dates:
    forecast_dates.append(time_i.date())

print(len(forecast_dates))
print(len(y_pred_future))


90
90


In [ ]:
df_forecast = pd.DataFrame({'Date':np.array(forecast_dates), 'Open':y_pred_future})
df_forecast['Date']=pd.to_datetime(df_forecast['Date'])


original = df[['Date', 'Open']]
original['Date']=pd.to_datetime(original['Date'])
original = original.loc[original['Date'] >= '2022-5-1']
print(df_forecast)


         Date        Open
0  2023-11-28  110.444687
1  2023-11-29  110.425476
2  2023-11-30  110.698196
3  2023-12-01  111.221771
4  2023-12-02  111.878708
..        ...         ...
85 2024-02-21  114.761246
86 2024-02-22  115.737488
87 2024-02-23  116.720192
88 2024-02-24  117.738586
89 2024-02-25  118.506294

[90 rows x 2 columns]


<ipython-input-139-47b72112e8b2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original['Date']=pd.to_datetime(original['Date'])
